# 言語理解（Language Understanding）

今後は、自然言語による音声や入力されたコマンドを理解するために、コンピュータがAIを利用できるようになることが期待されています。例えば、「ライトをつけて」「扇風機をつけて」などの音声コマンドで家庭内の機器を操作できるホームオートメーションシステムを導入し、AI搭載機器がそのコマンドを理解して適切な行動を取ることができるようにしたいと考えています。

![A robot listening](./images/language_understanding.jpg)

## オーサリングおよび予測リソースを作成する

MicrosoftのCognitive Serviceには、Language Understandingサービスが含まれています。これにより、**発話（utterances）** をベースとして **エンティティ（entities）** と、それに作用する **意図（intents）** を定義することができます。

Language Understandingサービスを利用するには、2種類のリソースが必要です。

- **作成（Authoring）** リソース: 言語モデルの定義、トレーニング、およびテストに使用されます。これは、Azureサブスクリプションの**Language Understanding - Authoring**リソースである必要があります。
- **予測**リソース: モデルを公開し、それを使用するクライアントアプリケーションからの要求を処理するために使用されます。これは、Azureサブスクリプション内の**Language Understanding**または**Cognitive Services**リソースのいずれかである必要があります。

Language Understandingアプリを公開するには、**Language Understanding**リソースまたは**Cognitive Services**リソースのどちらかを利用できますが、アプリの**オーサリング** には、それらとは別に **Language Understanding** リソースを作成する必要があります。

> **重要**： 作成リソースは、4つの**リージョン**（オーストラリア東部、西ヨーロッパ、米国西部、スイス北部）のいずれかで作成する必要があります。ヨーロッパまたはオーストラリアの作成リソースで作成されたモデルは、それぞれヨーロッパまたはオーストラリアの予測リソースにのみデプロイできます。米国の作成リソースで作成されたモデルは、ヨーロッパとオーストラリア以外のAzureのロケーションにある予測リソースにデプロイできます。作成と予測の場所のマッチングについては、[作成と公開のリージョン](https://docs.microsoft.com/azure/cognitive-services/luis/luis-reference-regions)を参照してください。

1. 新しいブラウザタブで、Azureポータル（https://portal.azure.com）を開き、Microsoftアカウントでサインインします。
2. **+ リソースの作成** をクリックして、**Language Understanding**を検索します。
3. サービスがリストから **Language Understanding** をクリックします。
4. **Language Understanding** の画面が表示されたら、**作成** をクリックします。
5. 次の画面では、以下の設定を行い **確認と作成**をクリックします。
   - **Create option**: Both
   
   - **Project Details**
     - **サブスクリプション**: *ご自身のサブスクリプション*
     - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
   - **Instance Details**
     - **Name**: *ご自身のサービスのユニークな名前*
   - **Authoring Resource**
     - **リージョン**: *好みのリージョンを選択*
     - **Authoring pricing tier**: F0
   - **Prediction Resource**
     - **Prediction location**: *Authoring Resource で設定したリージョンと同じリージョンを選択*
     - **Prediction pricing tier**: F0
6. 設定の検証が成功したら**作成**をクリックします。
7. しばらく待ってリソースが作成されると、作成用と予測用の2つのLanguage Understandingリソースがプロビジョニングされます。これらのリソースは、リソースを作成した**リソースグループに移動**することで確認できます。

### Language Understandingアプリを作成する

Language Understandingで自然言語理解を実装するには、アプリを作成し、エンティティ、意図、発話を追加して、アプリに理解させたいコマンドを定義します。

1. 新しいブラウザタブを開き、作成リソースを作成したリージョンのLanguage Understandingポータルを開きます。
    - 米国: [https://www.luis.ai](https://www.luis.ai)
    - ヨーロッパ: [https://eu.luis.ai](https://eu.luis.ai)
    - オーストラリア: [https://au.luis.ai](https://au.luis.ai)

2. お客様のAzureサブスクリプションに関連付けられたMicrosoftアカウントを使用してサインインします。初めてLanguage Understandingポータルにサインインした場合は、アカウントの詳細にアクセスする権限をアプリに付与する必要がある場合があります。次に、Azureサブスクリプションで作成したばかりの既存のLanguage Understanding作成リソースを選択したら **Confirm** をクリックして**Welcome**ステップを完了します。

3. **Converation Apps** のページが開きます。**+New app**をクリックします。
4. 以下の設定で会話用の新しいアプリを作成します。
   - **Name**: Home Automation
   - **Culture**: English (*このオプションが利用できない場合は空白にしてください。*)
   - **Description**: Simple home automation
   - **Prediction resource**: *前のステップで作成した Language Understanding予測リソース*

5. "How to create an effective LIUS app" のパネルが表示された場合は、それを閉じます。

### エンティティ(Entities)を作成する

**エンティティ**とは、言語モデルが識別して何かをすることができる**もの**のことです。このケースでは、言語理解アプリは、照明や扇風機など、オフィス内のさまざまな**デバイス**を制御するために使用されるので、アプリに対応させたいデバイスの種類のリストを含む**デバイス**エンティティを作成します。デバイスの種類ごとに、デバイスの名前（例：*light*）と、このタイプのデバイスを参照するために使用される可能性のあるSynomyms(同義語、例：*lamp*）を識別するサブリストを作成します。

1. アプリのLanguage Understandingページの左側のペインで、**Entities**をクリックします。次に、**Create**をクリックして、*device* という名前の新しいエンティティを作成し、**List**タイプを選択して、**Create**をクリックします。
2. **List itemsページ** で、**Normalized Values** の下に *light* と入力し、ENTERを押します。
3. *light*の値が追加された後、**Synonyms** に *lamp*と入力し、ENTERを押します。
4. 同様の手順で **fan** という名前の2つ目のリストアイテムを追加し、Synomyms **AC** を追加します。

> **注意** このラボでは、指示された通りの小文字または大文字のテキストを正確に使用してください（例: Light ではなく light と入力します)。また、余分なスペースを追加しないでください。

### 意図(Intents)を追加する

**意図**とは、1つまたは複数のエンティティに対して実行したいアクションのことで、例えば、ライトをつけたり、扇風機を止めたりすることができます。この場合、デバイスをオンにするための意図と、デバイスをオフにするための意図の2つを定義します。それぞれの意図には、その意図を示すためにどのような言葉が使われるかを示すサンプル**発話**を指定します。

> **注意** このラボでは、指示された通りに正確な小文字または大文字のテキストを入力してください（例: "turn the light on" と入力します。"Turn the light on” ではありません。) また、余分なスペースは追加しないでください。

1. 左側のペインで **Intents** をクリックします。次に、**Create** をクリックして、**switch_on** という名前のインテントを追加し、**Done** をクリックします。
2. **Examples** の見出しと **Example user input** の小見出しの下に、***turn the light on*** という発話を入力し、ENTERキーを押すと、この発話がリストに登録されます。
3. *turn the light on*という発話の中で、"light "という単語をクリックし、それを **device** エンティティの**light** 値に割り当てます。

![How to assign the word "light" to the entity value.](./images/assign_entity.jpg)

4. 意図 **switch_on** に、***turn the fan on*** というフレーズで2つ目の発話を追加します。そして、**device**エンティティの**fan**値に “fan"という単語を割り当てます。
5. 左側のペインで、**Intents** をクリックし、**Create** をクリックして、**switch_off** という名前の2つ目の意図を追加します。
6. 意図 **switch_off** の **Utterances**ページで、***turn the light off*** という発話を追加し、**device** エンティティの**light**値に ”light" という単語を割り当てます。
7. 意図 **switch_off** に、***turn the fan off*** というフレーズで2つ目の発話を追加します。そして、単語 “fan” を**device** エンティティの **fan** 値に接続します。

### 言語モデルのトレーニングとテスト

これで、エンティティ、意図、発話の形で提供されたデータを使って、アプリの言語モデルをトレーニングする準備が整いました。

1. アプリの **Language Understanding** ページの上部にある **Train** をクリックして、言語モデルをトレーニングします。
2. モデルのトレーニングが完了したら、**テスト** をクリックし、*テスト* ペインで以下のフレーズで予測される意図を確認します。
    * *switch the light on*
    * *turn off the fan*
    * *turn the lamp off*
    * *switch on the AC*
3. テストが終わったらテストペインを閉じます。
    
### モデルの公開とエンドポイントの設定

学習したモデルをクライアントアプリケーションで使用するには、クライアントアプリケーションが新しい発話を送信できるエンドポイントとしてモデルを公開する必要があります。そのエンドポイントを使って、意図やエンティティが予測されます。

1. 作成したアプリのLanguage Understandingページの上部で、**Publish** をクリックします。次に、**Production slot** を選択し、**Done** をクリックします。

2. モデルが公開されたら、アプリの言語理解ページの上部にある **MANAGE** をクリックします。次に、**Settings** タブで、あなたのアプリの **App ID** をメモします。これをコピーして、以下のコードに貼り付け、**YOUR_LU_APP_ID** を置き換えます。

3. **Azure Resources** タブで、予測リソースの **Primary key** と **Endpoint URL** をメモします。これらをコピーして、**YOUR_LU_KEY** と **YOUR_LU_ENDPOINT** を置き換えて以下のコードに貼り付けます。

4. セルの左側にある**Run cell** (&#9655;)ボタンをクリックして下のセルを実行し、プロンプトが表示されたら *turn the light on* というテキストを入力します。入力されたテキストは言語理解モデルによって解釈され、適切な画像が表示されます。

### **(!) 重要**
ウィンドウの上部に表示されているプロンプトを確認してください。ここでは、"turn the light on "と入力し、ENTERキーを押してください。


In [ ]:
from python_code import luis
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

try:
    # Set up API configuration
    luis_app_id = 'YOUR_LU_APP_ID'
    luis_key = 'YOUR_LU_KEY'
    luis_endpoint = 'YOUR_LU_ENDPOINT'
    # prompt for a command
    command = input('Please enter a command: \n')

    # get the predicted intent and entity (code in python_code.home_auto.py)
    action = luis.get_intent(luis_app_id, luis_key, luis_endpoint, command)

    # display an appropriate image
    img_name = action + '.jpg'
    img = Image.open(os.path.join("data", "luis" ,img_name))
    plt.axis('off')
    plt. imshow(img)
except Exception as ex:
    print(ex)

### (!) チェックイン
上記のセルを実行し、プロンプトが表示されたら *turn the light on* というフレーズを入力しましたか？プロンプトは、ウィンドウの上部に表示されます。 

上記のセルを再実行し、以下のフレーズを試してみてください。

* *turn on the light*
* *put the lamp off*
* *switch the fan on*
* *switch the light on*
* *switch off the light*
* *turn off the fan*
* *switch the AC on*

上記のセルを実行してクエスチョンマークの画像が表示された場合は、エンティティ、意図、または発話を作成する際に、指示されたものとは異なるテキストまたはスペースを入力して登録した可能性があります。

> **補足** 言語理解アプリから意図やエンティティを取得するためのコードが気になる方は、**python_code** フォルダ内の **luis.py** ファイルをご覧ください。

## 音声コントロールを追加する

ここまでテキストを分析する方法を見てきましたが、最近のAIシステムでは、音声認識によって人間がソフトウェアサービスとコミュニケーションできるようになってきています。これをサポートするために、**Speech** Cognitive Serviceでは、話し言葉をテキストに変換する簡単な方法を提供しています。

### Cognitive Servicesリソースを作成する

まだお持ちでない方は、以下の手順でAzureサブスクリプションに**Cognitive Services**リソースを作成してください。

> **補足** すでにCognitive Servicesリソースがある場合は、Azureポータルでその**クイックスタート**ページを開き、そのキーと場所を下のセルにコピーするだけでOKです。そうでない場合は、以下の手順で作成してください。

1. 新しいブラウザタブで、[https://portal.azure.com](https://portal.azure.com)のAzureポータルを開き、Microsoftアカウントでサインインします。
2. **&#65291;リソースを追加** ボタンをクリックし、*Cognitive Services* を検索し、以下の設定で **Cognitive Services** リソースを作成してください。
    - **サブスクリプション**: *ご自身のサブスクリプション*
    - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
    - **リージョン**: *利用可能なリージョンを選択（例:東日本）*
    - **名前**: *ユニークな名前を入力*
    - **価格レベル**: S0
    - **このボックスをオンにすることにより、以下のすべてのご契約条件を読み、同意したものとみなされます**: チェックを入れます。
3. デプロイが完了するのを待ちます。次に、Cognitive Serviceリソースにアクセスし、**クイック・スタート**ページで、キーとロケーションをメモします。これらは、クライアントアプリケーションからCognitive Serviceリソースに接続する際に必要となります。

### Cognitive Serviceリソースのキーと場所/地域の文字列を取得する

Cognitive Serviceリソースを利用するためには、クライアントアプリケーションはその認証キーと場所/地域の文字列が必要となります。

1. Azureポータルで、Cognitive Serviceリソースの**キーとエンドポイント**ページで、リソースの**キー1**をコピーして、**YOUR_COG_KEY**を置き換えて、以下のコードに貼り付けます。
2. リソースの**場所/地域** の文字列をコピーし、**YOUR_COG_LOCATION**を置き換えて以下のコードに貼り付けてください。
> **補足**  *キーとエンドポイント* ページを開き、このページから **場所/地域** の文字列をコピーしてください（例：_westus_）。Location フィールドの単語の間には、スペースを入れないでください。
3. 下のセルのコードを実行してください。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_location = 'YOUR_COG_LOCATION'
print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

以下のセルを実行してオーディオファイルから音声を書き起こし、Language Understandingアプリのコマンドとして使用します。

In [ ]:
import os
from python_code import luis
from azure.cognitiveservices.speech import SpeechConfig, SpeechRecognizer, AudioConfig
from playsound import playsound
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

try:   

    # Get spoken command from audio file
    file_name = 'light-on.wav'
    audio_file = os.path.join('data', 'luis', file_name)

    # Configure speech recognizer
    speech_config = SpeechConfig(cog_key, cog_location)
    audio_config = AudioConfig(filename=audio_file) # Use file instead of default (microphone)
    speech_recognizer = SpeechRecognizer(speech_config, audio_config)

    # Use a one-time, synchronous call to transcribe the speech
    speech = speech_recognizer.recognize_once()

    # Get the predicted intent and entity (code in python_code.home_auto.py)
    action = luis.get_intent(luis_app_id, luis_key, luis_endpoint, speech.text)

    # Get the appropriate image
    img_name = action + '.jpg'

    # Display image 
    img = Image.open(os.path.join("data", "luis" ,img_name))
    plt.axis('off')
    plt. imshow(img)
    playsound(audio_file)

except Exception as ex:
    print(ex)

上のセルのコードを修正して、**light-off.wav**オーディオファイルで試してみてください。

## さらに学ぶ

Language Understandingについての詳しい情報は、[Language Understanding (LUIS) のドキュメント](https://docs.microsoft.com/azure/cognitive-services/luis/)をご覧ください。